# Ancillary external photometry

This tutorial shows how to use external photometric data to perform astrometry calibrations on a Cube

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pykoala.data_container import Cube

cube = Cube.from_fits("products/Henize_2-10_cube_no_sky.fits.gz",
                        hdul_extensions_map={"INTENSITY": "INTENSITY", "VARIANCE": "VARIANCE"})

# Read the sky position and cube pixel size from the header
he2_10_ra, he2_10_dec = cube.wcs.celestial.wcs.crval[:2]
cube_pix_area = (cube.wcs.celestial.wcs.cdelt[0] * 3600)**2
print(f"Cube position: RA={he2_10_ra}, DEC={he2_10_dec}")
print("Cube pixel area (arcsec^2) ", cube_pix_area)

In [ ]:
from pykoala.photometry import query_image, PSQuery
from astropy import units as u
# Query r-band photometry from PS using a cutout with the size of the input cube
# + 30 additional arcseconds on every side. Save the fits in the current working directory
image_query_results = query_image(
    [cube], filters='i', im_extra_size_arcsec=30 * u.arcsec, im_output_dir='.')

# Alternatively, you can reload the image by doing:
# ps_filename = "ps_query_129.0629_n26.4126_i.fits"
# image_query_results = {"PS1.i": {}}
# image_query_results["PS1.i"]["ccd"] = PSQuery.read_ps_fits(ps_filename)
# image_query_results["PS1.i"]["pix_size"] = PSQuery.pixelsize_arcsec

In [ ]:
from pykoala.corrections.astrometry import AstrometryOffsetCorrection

astrometric_correction, results = AstrometryOffsetCorrection.from_external_image(
    cube, image_query_results["PS1.i"], filter_name="PANSTARRS_PS1.i")


In [ ]:
results["offset_fig"]

In [ ]:
cube = astrometric_correction.apply(cube)
# Save the final version of the cube
cube.to_fits("products/Henize_2-10_cube_astrometry.fits.gz")